In [74]:
import sys
sys.path.append("../../src")

import numpy as np
import pandas as pd

import lightgbm

from utils import ndcg_calculator
from LGBM_Rank import LGBMRank
from dataload import dataload, day_week_feature_engineering, train_label_split
from tqdm import tqdm

# dataload

In [75]:
path='../../data/'

test_answer_week, test_answer_month, \
df_train_week, df_train_month, \
sample_sumbission_week, sample_sumbission_month = dataload(path)

In [76]:
# # 전체 데이터 이용시
# df_history = pd.read_csv(path+'history_data.csv')

# ## 날짜 전처리
# df_history = df_history.assign(log_dt = pd.to_datetime(df_history.log_time//100, format="%Y%m%d%H%M"))
# df_history = df_history.assign(log_date = df_history.log_dt.dt.floor("D"))
# df_history = df_history.drop("log_time", axis=1)

# df_train_week = df_history.copy()

# # month 이용
# # df_train_week = df_train_month.copy()

# day_week feature engineering
- to split label, train data

In [77]:
df_train_week = day_week_feature_engineering(df_train_week)

log date min: 2022-03-01 00:00:00 log date max: 2022-04-23 00:00:00
min day: 0 max day: 53
min week: 0 max week: 7


# label & train split

In [78]:
# Label, train data split
df_train, label_df = train_label_split(df_train_week)

split last week: 7


In [79]:
n = 25

# prerpocess

In [80]:
personal_train = df_train.drop_duplicates(subset=['profile_id','album_id','ss_id'])
df_train = df_train.drop_duplicates(subset=['profile_id','album_id'])

In [81]:
# 라벨 유저 한정1
# customers = label_df.profileQ_id.unique()
# 전체 유저 한정
customers = df_train.profile_id.unique()

# MP

In [82]:
MP_df = df_train.album_id.value_counts().head(50).reset_index()
MP_df.columns = ['album_id','total_counts']
MP_df['join_col'] = 1

customer_df = df_train_week[['profile_id']].drop_duplicates()
customer_df['join_col'] = 1

In [83]:
MP_cand = customer_df.merge(MP_df, on='join_col').drop_duplicates(subset=['profile_id','album_id'])[['profile_id','album_id','total_counts']]

# general MP
- 마지막 1주, 2주의 MP를 각 유저마다 넣는다.

In [84]:
last_week_list = np.sort(df_train.week.unique())

In [85]:
print('중복제거 후 데이터 수:', len(df_train_week))

중복제거 후 데이터 수: 945518


In [86]:
# 마지막 6,5주 각각 MP를 10개 뽑음
last_week_ver1 = last_week_list[-1]
last_week_ver2 = last_week_list[-2]

MP_latest_ver1_df = df_train.query(f"week == {last_week_ver1}")

In [87]:
MP_df = MP_latest_ver1_df.groupby('album_id')['profile_id'].count().sort_values(ascending=False)
MP_df = MP_df.reset_index()
MP_df.columns = ['album_id','counts']
MP_candidate_df = MP_df[:10].copy()
MP_candidate_df['join_col'] = 1

In [88]:
# df_train_week 전체 유저 대상으로 후보군을 뽑을 것임
customer_df = df_train_week[df_train_week['profile_id'].isin(customers)][['profile_id']]
customer_df['join_col'] = 1
popular_articles_cand_ver1 = customer_df.copy()
popular_articles_cand_ver1 = popular_articles_cand_ver1.merge(MP_candidate_df, on="join_col")

popular_articles_cand_ver1.drop_duplicates(subset=['profile_id','album_id'],inplace=True)

In [89]:
MP_latest_ver2_df = df_train.query(f"week == {last_week_ver2}")

In [90]:
MP_df = MP_latest_ver2_df.groupby('album_id')['profile_id'].count().sort_values(ascending=False)
MP_df = MP_df.reset_index()
MP_df.columns = ['album_id','general_counts']
MP_candidate_df = MP_df[:10].copy()
MP_candidate_df['join_col'] = 1

In [91]:
customer_df = df_train_week[df_train_week['profile_id'].isin(customers)][['profile_id']]
customer_df['join_col'] = 1
popular_articles_cand_ver2 = customer_df.copy()
popular_articles_cand_ver2 = popular_articles_cand_ver2.merge(MP_candidate_df, on="join_col")

popular_articles_cand_ver2.drop_duplicates(subset=['profile_id','album_id'],inplace=True)

In [92]:
popular_articles_cand = pd.concat([popular_articles_cand_ver1, popular_articles_cand_ver2])
popular_articles_cand = popular_articles_cand.groupby(['profile_id','album_id'])['general_counts'].sum().reset_index()

# personal_MP

In [94]:
personal_MP_df = personal_train.groupby(['profile_id','album_id'])[['ss_id']].count().reset_index()
personal_MP_df.columns = ['profile_id','album_id','personal_counts']

In [95]:
# 서로 다른날 5회 이상 시청한 앨범만
personal_MP = personal_MP_df[personal_MP_df['personal_counts'] >= 5]
personal_MP = personal_MP.sort_values(by=['profile_id','personal_counts'],ascending=False)

In [96]:
# 상위 5개만 pick
head_df_list = []
# 전체 유저 대상으로 뽑기
for user_id in tqdm(customer_df.profile_id.unique()):
    personal_MP_user_len = len(personal_MP[personal_MP['profile_id']==user_id].head())
    random_choice_list = personal_MP.album_id.unique()
    if personal_MP_user_len <5:
        # 5개 아이템이 없는 경우 랜덤으로 없는 개수만 만큼 choice
        user_df = personal_MP[personal_MP['profile_id']==user_id]
        df = pd.DataFrame()
        random_choices = np.random.choice(random_choice_list, size=(5-personal_MP_user_len))
        df['profile_id'] = [user_id for _ in range(5-personal_MP_user_len)]
        df['album_id'] = random_choices
        df = pd.concat([user_df, df])
        head_df_list.append(df)
    else:
        head_df_list.append(personal_MP[personal_MP['profile_id']==user_id].head())
        
personal_MP_candidate = pd.concat(head_df_list)

100%|██████████████████████████████████████| 8057/8057 [00:08<00:00, 975.47it/s]


# MP_user_genre

In [98]:
# week, day, album_cnt, rank 컬럼 candidate 붙여야 함
# df_train

In [99]:
meta_df = pd.read_csv(path+'meta_data.csv')
meta_df = meta_df[['album_id','genre_mid','run_time','cast_1','cast_2','cast_3']]

/var/folders/tb/j9qmrzv54k73r5l100jxqll80000gn/T/ipykernel_62937/3810577724.py:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  meta_df = pd.read_csv(path+'meta_data.csv')


In [100]:
df_train_meta = df_train.merge(meta_df, on='album_id')

In [101]:
user_genre_df = df_train_meta.groupby(['profile_id','genre_mid']).count()['ss_id'].reset_index()
user_genre_df.columns = ['profile_id','genre_mid','genre_cnt']
user_genre_df = user_genre_df.groupby(['profile_id','genre_mid']).sum().reset_index().sort_values(by=['profile_id','genre_cnt'],ascending=False)

# 장르 선호도 피처 만들기
## 100이상 시청한 사람들만 percent
user_total_watch_dict = user_genre_df.groupby('profile_id')['genre_cnt'].sum()\
                        [user_genre_df.groupby('profile_id')['genre_cnt'].sum()>=100].to_dict()
# 전체 시청 피처 만들기
user_genre_df['user_genre_cnt'] = user_genre_df['profile_id'].apply(lambda x: user_total_watch_dict.get(x, None))
user_genre_df['user_genre_percent'] = user_genre_df['genre_cnt']/user_genre_df['user_genre_cnt']
user_genre_df.drop(columns=['user_genre_cnt'],inplace=True)
user_genre_df.dropna(subset=['user_genre_percent'],axis=0,inplace=True)

In [102]:
genre_top_items = {}
genre_count = df_train_meta['genre_mid'].value_counts()
for genre in genre_count.index:
    genre_top_items[genre] = list(df_train_meta[df_train_meta['genre_mid']==genre]['album_id'].value_counts().head(10).index)

In [103]:
df_list = []
for user_id in customer_df.profile_id.unique():
    user_genres = user_genre_df[user_genre_df['profile_id']== user_id].head(2)['genre_mid']
    
    df = pd.DataFrame()
    if len(user_genres) == 0:
        
        df['album_id'] = genre_top_items['노래율동']
        df['album_id'] = genre_top_items['TV만화']
        
    elif len(user_genres) == 1:
        genre_list_1 = genre_top_items[user_genres.values[0]]
        genre_list_2 = genre_top_items['노래율동']
        df['album_id'] = list(dict.fromkeys(np.append(genre_list_1,genre_list_2)))
        
    elif len(user_genres) == 2:
        genre_list_1 = genre_top_items.get(user_genres.values[0],[])
        genre_list_2 = genre_top_items.get(user_genres.values[1],[])
        df['album_id'] = list(dict.fromkeys(np.append(genre_list_1,genre_list_2)))

    df['profile_id'] = user_id
    df_list.append(df)

In [104]:
genre_candidate = pd.concat(df_list, ignore_index=True)
genre_candidate = genre_candidate[['profile_id','album_id']]

# ALS MF candidate

In [106]:
from ALS_MF import MF
model = MF(df_train_week,label_df)
als_candidate, item_factors_feature, user_factors_feature = model.mf_train()

  0%|          | 0/3 [00:00<?, ?it/s]

item vector shape: (20383, 200) user vector shape (8057, 200)


# apriori candidaate

In [107]:
from apriori import apriori_train, apriori_candidate

In [108]:
rules_confidence_item_week = apriori_train(df_train_week, 0.1, 0.8)
apriori_candidate, apriori_feature = apriori_candidate(df_train_week, rules_confidence_item_week)

In [109]:
apri_list = []
for i_list in apriori_candidate.album_id:
    apri_list.append(str(i_list))

In [110]:
pd.Series(apri_list).value_counts()

16     2530
15     2439
124    1988
65     1543
dtype: int64

# candidate merge

In [149]:
popular_articles_cand.album_id.nunique(), personal_MP_candidate.album_id.nunique(), genre_candidate.album_id.nunique()

(17, 4713, 167)

In [150]:
# MP_cand.drop(columns=['counts'],inplace=True)

In [162]:
candidate_0 = MP_cand
candidate_1 = popular_articles_cand[['profile_id','album_id']]
candidate_2 = personal_MP_candidate[['profile_id','album_id']]
candidate_3 = genre_candidate[['profile_id','album_id']]
candidate_4 = als_candidate.copy()
candidate_5 = apriori_candidate.copy()

cand = pd.concat([candidate_0, candidate_1, candidate_2, candidate_3, candidate_4, candidate_5])
# cand = candidate_0.copy()
cand.drop_duplicates(subset=['profile_id','album_id'],inplace=True)

In [163]:
candidate = pd.merge(cand, popular_articles_cand[['album_id','general_counts']].drop_duplicates(), how='left', on='album_id')
# personal_MP_df 달라짐
candidate = pd.merge(candidate,personal_MP_df, how='left', on=['profile_id','album_id'])
# candidate = pd.merge(candidate, apriori_feature, how="left", on=["profile_id","album_id"])

In [164]:
print('candidate 데이터 수:',len(candidate), 'cand 데이터 수:',len(cand))
print('candidate nunique: ', candidate.album_id.nunique())

candidate 데이터 수: 1215696 cand 데이터 수: 1215696
candidate nunique:  8691


# model preprocess

In [165]:
profile_df = pd.read_csv(path+'profile_data.csv')

In [166]:
candidate_add_features = pd.merge(candidate,profile_df, how='left', on='profile_id')
candidate_add_features = pd.merge(candidate_add_features,meta_df.drop_duplicates('album_id'), how='left', on='album_id')

In [167]:
# columns = ['sex','pr_interest_keyword_cd_1','pr_interest_keyword_cd_2','pr_interest_keyword_cd_3','ch_interest_keyword_cd_1','ch_interest_keyword_cd_2','ch_interest_keyword_cd_3',\
# 'genre_mid','cast_1','cast_2','cast_3']
# from sklearn.preprocessing import LabelEncoder
# for col in columns:
#     LE = LabelEncoder()
#     candidate_add_features[col] = LE.fit_transform(candidate_add_features[col])

In [168]:
candidate_add_features = candidate.copy()
# candidate_add_features = candidate_add_features.drop(columns=['personal_counts','pr_interest_keyword_cd_1',\
#                                                               'pr_interest_keyword_cd_2','pr_interest_keyword_cd_3'\
#                                                               ,'ch_interest_keyword_cd_1','ch_interest_keyword_cd_2'\
#                                                               ,'ch_interest_keyword_cd_3',\
#                                                                 'genre_mid','cast_1','cast_2','cast_3'])

In [169]:
candidate_add_features = pd.merge(candidate_add_features, item_factors_feature, how="left", on="album_id")
candidate_add_features = pd.merge(candidate_add_features, user_factors_feature, how="left", on="profile_id")

In [170]:
train_df = pd.merge(candidate_add_features, label_df, how='left', on=['profile_id','album_id'])
train_df['rating'] = train_df['rating'].fillna(0)
# train_df = train_df.drop(columns="personal_counts")

# model

In [171]:
train_df.rating.value_counts()

0.0    1175038
1.0      40658
Name: rating, dtype: int64

In [172]:
lgbmrank = LGBMRank(train_df, mode='week', model_params={'n_estimators':5})
X_train, sample_sumbission = lgbmrank.valid_evaluation()

                 feature_importances
total_counts              813.035889
general_counts           4894.915817
personal_counts        240068.768776
0_x                         0.000000
1_x                         0.000000
...                              ...
195_y                       0.000000
196_y                       0.000000
197_y                       0.000000
198_y                       0.000000
199_y                      86.140602

[403 rows x 1 columns]
week performance
lgbm ndcg: 0.18717950676758846


In [161]:
lgbmrank = LGBMRank(train_df, mode='week', model_params={'n_estimators':5})
X_train, sample_sumbission = lgbmrank.valid_evaluation()

                feature_importances
total_counts            2400.180016
general_counts          1032.932030
0_x                        0.000000
1_x                        0.000000
2_x                        0.000000
...                             ...
195_y                      0.000000
196_y                      0.000000
197_y                      0.000000
198_y                      0.000000
199_y                      0.000000

[402 rows x 1 columns]
week performance
lgbm ndcg: 0.07670088572681977


In [124]:
# cold user MP로 채우기 
MP_list = MP_cand.album_id.unique()

sample_sumbission_cold = sample_sumbission.copy()
sample_sumbission_cold['album_id'] = sample_sumbission_cold['album_id']\
                                        .apply(lambda x: list(dict.fromkeys(np.append(x, MP_list)))[:n])

In [125]:
ndcg_calculator(sample_sumbission_cold, test_answer_week, n)

0.18477461088544422

In [126]:
set_list = {}
for list_i in sample_sumbission.album_id:
    set_list[str(list_i)] = 0
len(set_list)

2187

# Evaluation

### experiment
- num_leaves= 20, learning_rate=0.005, n_estimators:5
- 어느 cand 중요한 지 판단
- popular_articles_cand :(10,10) , personal_MP_candidate:(5), genre_candidate: max(10,10)

### week
- week ndcg score : 0.05611031122402796
- popular_articles_cand week: ndcg 0.028022766659891125 (total unique item: 12)
- personal_MP_candidate week: ndcg 0.02896244104871668  (total unique item: 4303)
- genre_candidate week:       ndcg 0.01579135161487282  (total unique item: 157)
- (popular_articles_cand, personal_MP_candidate) week: ndcg 0.05206722084683086  (total unique item: 4303)
- (popular_articles_cand, genre_candidate) week: 0.03809584406788012 (total unique item: 161)
- (personal_MP_candidate, genre_candidate) week: 0.041085241338965385 (total unique item: 4338)

### month
- month ndcg score : 0.08036130090004782
- popular_articles_cand month: ndcg 0.05945057771810242 (total unique item: 15)
- personal_MP_candidate month: ndcg 0.028477345633295483  (total unique item: 2826)
- genre_candidate       month: ndcg 0.017431233023063257  (total unique item: 150)
- (popular_articles_cand, personal_MP_candidate) month: ndcg 0.05206722084683086  (total unique item: 2826)
- (popular_articles_cand, genre_candidate) month: 0.0680887232688415 (total unique item: 155)
- (personal_MP_candidate, genre_candidate) month: 0.041085241338965385 (total unique item: 2872)

In [127]:
def evaluation(
            X_train:pd.DataFrame(), 
            sumbission:pd.DataFrame(), 
            n:int, 
            MP_cand:pd.DataFrame()
            )->pd.DataFrame():
    
    MP_list = MP_cand.album_id.unique()
    
    # each user pred 25 items
    lgbm_sub_df = X_train.sort_values(by='pred', ascending=False).groupby('profile_id').head(n)
    lgbm_user_items_dict = lgbm_sub_df.groupby('profile_id')['album_id'].unique().to_dict()
    sumbission['predicted_list'] = sumbission['profile_id']\
                                            .apply(lambda x: lgbm_user_items_dict.get(x, []))
    
    # cold start user file MP_list top25
    sumbission_cold = sumbission.copy()
    sumbission_cold['predicted_list'] = sumbission_cold['predicted_list']\
                                            .apply(lambda x: list(dict.fromkeys(np.append(x, MP_list)))[:n])
    
    return sumbission, sumbission_cold

In [128]:
submission = pd.read_csv(path + 'sample_submission.csv')
sumbission_pred, sumbission_cold = evaluation(X_train, submission, n, MP_cand)

In [129]:
set_list = {}
for list_i in sumbission_cold.predicted_list:
    set_list[str(list_i)] = 0

In [130]:
print('같은 추천을 받은 유저 수:', 8311 - len(set_list))

같은 추천을 받은 유저 수: 253


In [131]:
sumbission_cold

,profile_id,predicted_list
0,3,"[16, 15, 19, 18, 17, 33, 125, 124, 241, 1888, ..."
1,5,"[264, 16, 38, 15, 241, 190, 178, 67, 136, 18, ..."
2,7,"[237, 343, 2796, 1901, 1999, 2865, 2489, 242, ..."
3,12,"[352, 353, 2029, 19, 18, 17, 224, 416, 0, 39, ..."
4,16,"[339, 124, 65, 536, 33, 2492, 2473, 2491, 505,..."
...,...,...
8306,33022,"[124.0, 16.0, 125.0, 241.0, 65.0, 15.0, 38.0, ..."
8307,33023,"[124.0, 16.0, 125.0, 241.0, 65.0, 15.0, 38.0, ..."
8308,33026,"[124.0, 16.0, 125.0, 241.0, 65.0, 15.0, 38.0, ..."
8309,33027,"[124.0, 16.0, 125.0, 241.0, 65.0, 15.0, 38.0, ..."


In [132]:
# 제출 조건 충족 확인
assert submission.profile_id.nunique() == sumbission_cold.profile_id.nunique()
for pred_list in sumbission_cold.predicted_list:
    assert len(pred_list) == 25

In [133]:
sumbission_cold

,profile_id,predicted_list
0,3,"[16, 15, 19, 18, 17, 33, 125, 124, 241, 1888, ..."
1,5,"[264, 16, 38, 15, 241, 190, 178, 67, 136, 18, ..."
2,7,"[237, 343, 2796, 1901, 1999, 2865, 2489, 242, ..."
3,12,"[352, 353, 2029, 19, 18, 17, 224, 416, 0, 39, ..."
4,16,"[339, 124, 65, 536, 33, 2492, 2473, 2491, 505,..."
...,...,...
8306,33022,"[124.0, 16.0, 125.0, 241.0, 65.0, 15.0, 38.0, ..."
8307,33023,"[124.0, 16.0, 125.0, 241.0, 65.0, 15.0, 38.0, ..."
8308,33026,"[124.0, 16.0, 125.0, 241.0, 65.0, 15.0, 38.0, ..."
8309,33027,"[124.0, 16.0, 125.0, 241.0, 65.0, 15.0, 38.0, ..."


In [134]:
# sumbission_cold.to_csv('lgbm_candidate_submission_ver5_alsvector.csv', index=False)